In [7]:
using ITensors, ITensorMPS
using CUDA:cu

In [8]:
function make_coupling_matrix(Nx, Ny)
    # uniformly sample the coupling strength from [0, 2]
    coupling_matrix = zeros(Nx, Ny)
    for i in 1:Nx
        for j in i+1:Ny
            prob = 1 # rand()
            if j - i > 1
                if prob > 0.5
                    coupling_matrix[i, j] = coupling_matrix[j, i] = 0.0
                elseif coupling_matrix[i, j - 1] == 0.0
                    coupling_matrix[i, j] = coupling_matrix[j, i] = 0.0
                else
                    coupling_matrix[i, j] = coupling_matrix[j, i] = 2.0 * (rand() + 1e-6)
                end
            else 
                coupling_matrix[i, j] = coupling_matrix[j, i] = 2.0 * rand()
            end
        end
    end
    return coupling_matrix
end;

In [9]:
function make_coupling_strength(Nx, Ny)
    N = 2 * Nx * Ny - Nx - Ny
    coupling_strength = [2.0 * (rand() + 1e-6) for i in 1:N]
    return coupling_strength
end;

In [10]:
using ITensors, ITensorMPS
function heisenberg_2d_huang(Nx, Ny, spin, nsweeps, coupling_strength, maxdim, cutoff, device=identity)
    N = Nx * Ny
    J = coupling_strength
    sites = siteinds("S=$spin", N; conserve_qns=true)
    lattice = square_lattice(Nx, Ny; yperiodic=false)
    i = 1
    os = OpSum()
    for b in lattice
        os += J[i] / 2, "S+", b.s1, "S-", b.s2
        os += J[i] / 2, "S-", b.s1, "S+", b.s2
        os += J[i], "Sz", b.s1, "Sz", b.s2
        i += 1
    end
    H = MPO(os, sites)
    # H = map(os -> device(MPO(Float32, os, sites)), os)
    state = [isodd(n) ? "Up" : "Dn" for n=1:N]
    # psi0 = device(complex.(MPS(sites, state)))
    psi0 = MPS(sites, state)
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff)
    
    return energy, psi
end;

In [11]:
using ITensors, ITensorMPS
function heisenberg_2d(Nx, Ny, spin, nsweeps, coupling_strength, maxdim, cutoff, device=identity)
    N = Nx * Ny
    J = coupling_strength
    sites = siteinds("S=$spin", N; conserve_qns=true)
    lattice = square_lattice(Nx, Ny; yperiodic=false)
    
    os = OpSum()
    for b in lattice
        Ji = J[cld(b.s1, Ny) , mod(b.s1, Ny) == 0 ? Ny : mod(b.s1, Ny)]
        Jj = J[cld(b.s2, Ny) , mod(b.s2, Ny) == 0 ? Ny : mod(b.s2, Ny)]
        Jij = max(Ji, Jj)
        os += Jij / 2, "S+", b.s1, "S-", b.s2
        os += Jij / 2, "S-", b.s1, "S+", b.s2
        os += Jij, "Sz", b.s1, "Sz", b.s2
    end
    H = MPO(os, sites)
    # H = map(os -> device(MPO(Float32, os, sites)), os)
    state = [isodd(n) ? "Up" : "Dn" for n=1:N]
    # psi0 = device(complex.(MPS(sites, state)))
    psi0 = MPS(sites, state)
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff)
    
    return energy
end;

In [12]:
spin = "1/2"
nsweeps = 5
coupling_strength = make_coupling_matrix(5, 5)
maxerr = 1E-4
# coupling_strength = 0.5
maxdim = [20, 60, 80, 100, 100]
cuttoff = 1E-5
# energy = heisenberg_2d(Nx, Ny, spin, nsweeps, coupling_strength, maxdim, cuttoff)
coupling_strength

5×5 Matrix{Float64}:
 0.0      1.19539  0.0      0.0       0.0
 1.19539  0.0      0.20912  0.0       0.0
 0.0      0.20912  0.0      1.46237   0.0
 0.0      0.0      1.46237  0.0       0.926889
 0.0      0.0      0.0      0.926889  0.0

In [13]:
samples_num = 100
system_size = 20

20

In [15]:
Nx = 4
Ny = 5
spin = "1/2"
nsweeps = 5
coupling_strength = make_coupling_strength(Nx, Ny)
maxerr = 1E-4
maxdim = [20, 60, 80, 100, 100]
cuttoff = 1E-5
energy = heisenberg_2d_huang(Nx, Ny, spin, nsweeps, coupling_strength, maxdim, cuttoff);

After sweep 1 energy=-12.050660970627035  maxlinkdim=4 maxerr=8.51E-06 time=0.042
After sweep 2 energy=-12.91876505439032  maxlinkdim=12 maxerr=7.64E-06 time=0.100
After sweep 3 energy=-14.149201332098068  maxlinkdim=35 maxerr=9.84E-06 time=0.145
After sweep 4 energy=-14.408564247138232  maxlinkdim=58 maxerr=9.71E-06 time=0.197
After sweep 5 energy=-14.430187985475541  maxlinkdim=72 maxerr=9.82E-06 time=0.243


In [72]:
function heisenberg_2d_dataset(system_size, samples_num, spin, nsweeps, maxdim, cuttoff)
    dataset = []
    coupling_matrix_list = [make_coupling_matrix(system_size, system_size) for i in 1:samples_num]
    energy_list = [heisenberg_2d(system_size, system_size, spin, nsweeps, coupling_strength, maxdim, cuttoff) for coupling_strength in coupling_matrix_list]
    push!(dataset, (coupling_matrix_list, energy_list))
    return dataset
end;

In [77]:
# save the dataset
write_dataset_to_csv(dataset, "heisenberg_2d/n100_X(coupling)_y(energy)_q20.csv")

"heisenberg_2d/n100_X(coupling)_y(energy)_q20.csv"

In [19]:
function heisenberg_1d(N, spin, nsweeps, maxdim, cutoff)
    sites = siteinds("S=$spin", N)
    os = OpSum()
    for i=1:N-1
        os += 1 / 2, "S+", i, "S-", i+1
        os += 1 / 2, "S-", i, "S+", i+1
        os += 1, "Sz", i, "Sz", i+1
    end
    H = MPO(os, sites)
    
    psi0 = MPS(sites, n -> isodd(n) ? "Up" : "Dn")
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff)
    return energy, psi, H
end


heisenberg_1d (generic function with 1 method)

In [20]:
N = 12
spin="1/2"
nsweeps = 5
maxdim = [10,20,100,100,200]
cutoff = [1E-10]
energy, psi, H = heisenberg_1d(N, spin, nsweeps, maxdim, cutoff);

After sweep 1 energy=-5.0924746961312275  maxlinkdim=4 maxerr=7.08E-16 time=11.831
After sweep 2 energy=-5.142067911316417  maxlinkdim=16 maxerr=6.17E-11 time=0.013
After sweep 3 energy=-5.142090631641765  maxlinkdim=23 maxerr=9.41E-11 time=0.038
After sweep 4 energy=-5.142090631668938  maxlinkdim=23 maxerr=8.25E-11 time=0.018
After sweep 5 energy=-5.142090631668938  maxlinkdim=23 maxerr=8.24E-11 time=0.020


In [43]:
energy

-5.142090631668938

In [21]:
function compute_expectation_value(psi, H)
    ex_W = inner(psi', H, psi)
    return ex_W
end;

In [22]:
# computing entanglement entropy for 1-dimension model
function compute_entropy_1d(psi, b)
    psi = orthogonalize(psi, b)
    U,S,V = svd(psi[b], (linkinds(psi, b-1)..., siteinds(psi, b)...))
    SvN = 0.0
    for n=1:dim(S, 1)
        p = S[n,n]^2 # probability of n-th Schmidt coefficient
        SvN -= p * log(p) # von Neumann entropy
    end
    return SvN
end;


# computing correlation function
function compute_correlation_pauliz(psi)
    zzcorr = correlation_matrix(psi, "Sz", "Sz")
    
    # normalize the correlation matrix
    dig = diag(zzcorr)
    dig = sqrt.(dig)
    for i=1:length(dig)
        zzcorr[i,:] = zzcorr[i,:] ./ dig[i]
        zzcorr[:,i] = zzcorr[:,i] ./ dig[i]
    end

    return zzcorr
end;


In [58]:
x = compute_correlation_pauliz(psi)

12×12 Matrix{Float64}:
  1.0        -0.875037    0.264345   …  -0.0904036   0.0452814  -0.068897
 -0.875037    1.0        -0.387624       0.0597471  -0.0300538   0.0452814
  0.264345   -0.387624    1.0           -0.120865    0.0597472  -0.0904036
 -0.29411     0.222159   -0.764812       0.0924652  -0.0463522   0.0682842
  0.145969   -0.118818    0.248351      -0.168161    0.0804499  -0.119988
 -0.172313    0.119348   -0.272552   …   0.139509   -0.0691014   0.0968687
  0.0968687  -0.0691014   0.139509      -0.272552    0.119348   -0.172313
 -0.119988    0.0804499  -0.168161       0.248351   -0.118818    0.145969
  0.068284   -0.046352    0.0924652     -0.764812    0.222159   -0.29411
 -0.0904036   0.0597471  -0.120865       1.0        -0.387624    0.264345
  0.0452814  -0.0300538   0.0597472  …  -0.387624    1.0        -0.875037
 -0.068897    0.0452814  -0.0904036      0.264345   -0.875037    1.0

In [59]:
import CSV
import DataFrames
# cha
vec(x)
# a = Float32.(vec(x));

144-element Vector{Float64}:
  1.0000000000000002
 -0.8750367881118843
  0.26434543693100016
 -0.2941103904610836
  0.14596929888133
 -0.17231320446017406
  0.09686866163343585
 -0.11998788896175688
  0.06828402960932692
 -0.09040355084903764
  ⋮
  0.06828417311987559
 -0.11998795321985982
  0.09686866399598784
 -0.1723132083934401
  0.14596930010656733
 -0.2941103899358733
  0.2643454365601726
 -0.8750367870297763
  1.0

In [60]:
str_vec = "[" * join(a, ", ") * "]"
str_vec

"[1.0, -0.8750368, 0.26434544, -0.2941104, 0.1459693, -0.1723132, 0.096868664, -0.11998789, 0.06828403, -0.09040355, 0.045281384, -0.06889699, -0.8750368, 1.0, -0.38762358, 0.22215857, -0.11881778, 0.11934848, -0.069101445, 0.08044987, -0.046352003, 0.059747126, -0.03005" ⋯ 1158 bytes ⋯ "030053832, 0.05974722, -0.046352174, 0.080449946, -0.06910144, 0.11934849, -0.11881778, 0.22215857, -0.3876236, 1.0, -0.8750368, -0.06889699, 0.045281384, -0.09040363, 0.06828418, -0.11998795, 0.096868664, -0.17231321, 0.1459693, -0.2941104, 0.26434544, -0.8750368, 1.0]"

In [39]:
# covert x to an array with comma to split neighboring datapoint 
x = reshape(x, length(x), 1)
x = join(x, ",")
x = split(x, ",")
x = [parse(Float64, x) for x in x]
x

144-element Vector{Float64}:
  1.0000000000000002
 -0.8750367881118843
  0.26434543693100016
 -0.2941103904610836
  0.14596929888133
 -0.17231320446017406
  0.09686866163343585
 -0.11998788896175688
  0.06828402960932692
 -0.09040355084903764
  ⋮
  0.06828417311987559
 -0.11998795321985982
  0.09686866399598784
 -0.1723132083934401
  0.14596930010656733
 -0.2941103899358733
  0.2643454365601726
 -0.8750367870297763
  1.0

In [228]:
for i=1:12
    println(compute_entropy_1d(psi, i))
end

0.6931471805599452
0.414031145052179
0.7293379459975349
0.5114291012403798
0.7481119950888124
0.5368332154956549
0.7481119977360337
0.5114291059314526
0.7293379506858658
0.41403114778527816
0.6931471805599456
1.110223024625156e-15


In [237]:
include("mps_utils.jl")

compute_renyi_entropy_1d (generic function with 2 methods)

In [240]:
for i=1:12
    println(compute_renyi_entropy_1d(psi, i))
end

(1, 0.6931471805599448)
(2, 0.193260729470899)
(3, 0.7038391814469086)
(4, 0.2574983003332026)
(5, 0.710599103635078)
(6, 0.27523998137230477)
(7, 0.7105991041226506)
(8, 0.25749830134819246)
(9, 0.7038391829418945)
(10, 0.19326073119404186)
(11, 0.6931471805599476)
(12, 2.2204460492503154e-15)


In [262]:
energy, psi = heisenberg_2d_huang(5, 5, "1/2", 6, make_coupling_strength(5, 5), [20, 60, 80, 100, 100], 1E-5);

After sweep 1 energy=-13.49508378866567  maxlinkdim=4 maxerr=5.54E-06 time=0.081
After sweep 2 energy=-13.851244099950215  maxlinkdim=13 maxerr=8.82E-06 time=0.088
After sweep 3 energy=-13.933468914029456  maxlinkdim=18 maxerr=9.36E-06 time=0.125
After sweep 4 energy=-13.934164611120474  maxlinkdim=19 maxerr=9.36E-06 time=0.119
After sweep 5 energy=-13.934168163970948  maxlinkdim=19 maxerr=7.69E-06 time=0.103
After sweep 6 energy=-13.934168166136507  maxlinkdim=19 maxerr=7.70E-06 time=0.120


In [155]:
energy, psi = heisenberg_1d(12, "1", 5, [10,20,100,100,200], 1E-10)

After sweep 1 energy=-15.6026723305666  maxlinkdim=9 maxerr=1.17E-15 time=0.024
After sweep 2 energy=-15.654961849544552  maxlinkdim=20 maxerr=6.72E-06 time=0.071
After sweep 3 energy=-15.673064469227194  maxlinkdim=98 maxerr=9.87E-11 time=0.420
After sweep 4 energy=-15.673988951856119  maxlinkdim=96 maxerr=1.23E-10 time=0.707
After sweep 5 energy=-15.674010100017021  maxlinkdim=81 maxerr=9.94E-11 time=0.365


(-15.674010100017021, MPS
[1] ((dim=3|id=753|"Link,l=1"), (dim=3|id=897|"S=1,Site,n=1"))
[2] ((dim=9|id=361|"Link,l=2"), (dim=3|id=982|"S=1,Site,n=2"), (dim=3|id=753|"Link,l=1"))
[3] ((dim=3|id=806|"S=1,Site,n=3"), (dim=27|id=343|"Link,l=3"), (dim=9|id=361|"Link,l=2"))
[4] ((dim=3|id=288|"S=1,Site,n=4"), (dim=61|id=789|"Link,l=4"), (dim=27|id=343|"Link,l=3"))
[5] ((dim=3|id=117|"S=1,Site,n=5"), (dim=74|id=754|"Link,l=5"), (dim=61|id=789|"Link,l=4"))
[6] ((dim=3|id=974|"S=1,Site,n=6"), (dim=81|id=484|"Link,l=6"), (dim=74|id=754|"Link,l=5"))
[7] ((dim=3|id=411|"S=1,Site,n=7"), (dim=76|id=361|"Link,l=7"), (dim=81|id=484|"Link,l=6"))
[8] ((dim=3|id=819|"S=1,Site,n=8"), (dim=62|id=563|"Link,l=8"), (dim=76|id=361|"Link,l=7"))
[9] ((dim=3|id=185|"S=1,Site,n=9"), (dim=27|id=853|"Link,l=9"), (dim=62|id=563|"Link,l=8"))
[10] ((dim=3|id=101|"S=1,Site,n=10"), (dim=9|id=434|"Link,l=10"), (dim=27|id=853|"Link,l=9"))
[11] ((dim=3|id=704|"S=1,Site,n=11"), (dim=3|id=68|"Link,l=11"), (dim=9|id=434|"Link

In [263]:
compute_correlation_pauliz(psi)

25×25 Matrix{Float64}:
  1.0       -0.853308   0.617832  …   0.42981   -0.271353   0.353383
 -0.853308   1.0       -0.764524     -0.347341   0.219287  -0.285578
  0.617832  -0.764524   1.0           0.479724  -0.302865   0.394421
 -0.764524   0.617832  -0.853308     -0.562194   0.35493   -0.462226
  0.764524  -0.617832   0.853308      0.562194  -0.35493    0.462226
 -0.4331     0.35      -0.483396  …  -0.318482   0.201067  -0.261851
  0.412037  -0.332978   0.459887      0.30299   -0.191288   0.249114
 -0.692249   0.559425  -0.77264      -0.509051   0.321377  -0.418534
  0.610006  -0.492962   0.680846      0.448458  -0.283221   0.368689
 -0.738961   0.597173  -0.824776     -0.543398   0.343062  -0.446773
  ⋮                               ⋱                        
  0.470846  -0.380503   0.525525      0.365045  -0.231278   0.290436
 -0.665972   0.53819   -0.743312     -0.513752   0.31943   -0.419627
  0.427643  -0.34559    0.477305      0.324796  -0.252024   0.31248
 -0.442811   0.357847

In [266]:
[compute_renyi_entropy_1d(psi, b) for b in 1:25]

25-element Vector{Tuple{Int64, Float64}}:
 (1, 0.23288043602723432)
 (2, 0.14610476965048264)
 (3, -0.0)
 (4, -0.0)
 (5, -0.0)
 (6, 0.41481953713339476)
 (7, 0.028302369728264807)
 (8, 0.07176403133444796)
 (9, 0.13468286974488564)
 (10, 0.10150579502962617)
 ⋮
 (17, 0.19681027329646508)
 (18, 0.17684523636247154)
 (19, 0.442338421344833)
 (20, 0.16355474020402883)
 (21, 0.3832500278778335)
 (22, 0.3498126293465143)
 (23, 0.21994848684181517)
 (24, 0.49951215521276676)
 (25, -2.664535259100372e-15)